In [61]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from copy import deepcopy
from scipy.optimize import curve_fit


from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence
import experiments as meas
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=3)
import qutip as qt
import matplotlib.style as style
style.use('S:\Connie\prx.mplstyle')

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

In [3]:
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

In [4]:
def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]
    # SI_to_SLR = [psis['000'], psis['010'], psis['100'], psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    # print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)


In [5]:
tomo_qubits = [0, 2, 3]

# Save paths

In [6]:
save_plot_path = 'S:\\QRAM\\qram_4QR2\\plots\\paper'

# Formatting

In [7]:
color_switch = '#ff7f50'
color_input = '#468499'
color_q2 = '#E16F8F'
color_basis = '#66CDAA'

def CQ_from_init(init_state):
    _init_state = init_state.replace('|', '')
    q0, q1 = _init_state[:-1].split('>')
    if q0 == '0+1' or q0=='0+i':
        if q1 != '0+1' and q1 !='0+i':
            return 'QsCi'
        return 'QsQi'

    else: 
        if q1 == '0+1' or q1=='0+i':
            return 'CsQi'
        return 'CsCi'

def color_from_cq(cq):
    if cq == 'QsCi':
        return color_switch
    elif cq == 'QsQi':
        return color_q2
    elif cq == 'CsQi':
        return color_input
    elif cq == 'CsCi':
        return color_basis

def latex_from_cq(cq):
    return cq

    if cq == 'QsCi':
        return '$Q_\mathrm{switch}C_\mathrm{input}$'
    elif cq == 'QsQi':
        return '$Q_\mathrm{switch}Q_\mathrm{input}$'
    elif cq == 'CsQi':
        return '$C_\mathrm{switch}Q_\mathrm{input}$'
    elif cq == 'CsCi':
        return '$C_\mathrm{switch}C_\mathrm{input}$'


# Import 2q tomo

In [8]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_250119\\202502111059_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_250119\202502111059_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


# Load evolv mats

In [9]:
evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"

In [10]:
print('Will save evol mats to path', evol_mats_path)
qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'

qA, qB, qC = tomo_qubits
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})

Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats
Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


# Get fids grids and n_tomo

In [107]:
use_init_2q_state = True
plot = True

data_path = "S:\QRAM\qram_4QR2\data\data_250119\\"

fids_grids_filepath_1protocol = "202502120248_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_3protocol = "202502120636_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_5protocol = "202502121025_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_7protocol = "202502121415_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_9protocol = "202502130312_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_11protocol = "202502130703_ntomocorrected_3Q_023_fids_grids.npz"

fids_grids_filepaths = [fids_grids_filepath_1protocol, fids_grids_filepath_3protocol, fids_grids_filepath_5protocol, fids_grids_filepath_7protocol, fids_grids_filepath_9protocol, fids_grids_filepath_11protocol]

In [108]:
def get_fids_grids(fids_grids_filepath):
    # RETRIEVING MEASURED n_tomo_corrected and fids grids
    print(f"Using fids_grids_dict from file {fids_grids_filepath}")
    fids_grids_dict = dict()

    with np.load(fids_grids_filepath, allow_pickle=True) as npzfile:
        for key in npzfile.keys():
            fids_grids_dict.update({key:npzfile[key]})

    init_states = fids_grids_dict["init_states"]

    if 'saved_files' in init_states: init_states = init_states[:-1]

    print(f'fids_grids_dict retrieved with init_states\n{init_states}')
    all_configs = fids_grids_dict["all_configs"].item() #[0]
    # print(all_configs.keys())

    cfg = AttrDict(all_configs[init_states[0]])
    play_pulses = cfg.expt.play_pulses
    tomo_qubits = cfg.expt.tomo_qubits

    print("play_pulses", play_pulses)

    return fids_grids_dict

In [109]:
fids_grids_dict = get_fids_grids(os.path.join(data_path, fids_grids_filepath_1protocol))

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_250119\202502120248_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 3, 4]


Pick best overall phi

In [110]:
def get_optimized_phis(fids_grids_dict, use_init_2q_state, use_ZZ_correction, show_fid_grids=False):
    init_states = fids_grids_dict["init_states"]
    check_init_states = init_states
    
    phis = fids_grids_dict["phis"]

    inner_sweep = phis[2]
    outer_sweep = phis[1]
    y_sweep = outer_sweep
    x_sweep = inner_sweep

    if use_ZZ_correction and use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_ZZ_vs_meas"]
    elif use_ZZ_correction and not use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_ZZ_vs_ideal"]
    elif not use_ZZ_correction and use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_base_vs_meas"]
    elif not use_ZZ_correction and not use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_base_vs_ideal"]

    best_avg_fid_avg_opt = 0
    best_fids_avg_opt = []
    best_phis_avg_opt = [0, 0, 0]
    best_phi_indices = [0, 0, 0]
    individual_best_fids = np.zeros((len(init_states)))
    individual_best_phis = np.zeros((len(init_states), 3))
    individual_best_phis_indices = np.zeros((len(init_states), 3), dtype=int)
    for iphi0, phi0 in enumerate(phis[0]):
        for iphi1, phi1 in enumerate(phis[1]):
            for iphi2, phi2 in enumerate(phis[2]):
                avg_fid = 0
                for i_state, init_state in enumerate(init_states):
                    if init_state not in check_init_states: continue
                    fid = fids_grids[i_state][iphi0, iphi1, iphi2]
                    avg_fid += fid
                    if fid > individual_best_fids[i_state]:
                        individual_best_fids[i_state] = fid
                        individual_best_phis[i_state, :] = [phi0, phi1, phi2]
                        individual_best_phis_indices[i_state, :] = [iphi0, iphi1, iphi2]
                avg_fid /= len(check_init_states)
                if avg_fid > best_avg_fid_avg_opt:
                    best_phis_avg_opt = [phi0, phi1, phi2]
                    best_phi_indices = [iphi0, iphi1, iphi2]
                    best_avg_fid_avg_opt = avg_fid

    if show_fid_grids:
        vmax = 1.0
        for i_state in range(len(init_states)):
            # print(init_states[i_state])
            # print(f'best phi Q0 {best_phis_avg_opt[0]}')
            plt.figure()
            # plt.title(f'Fidelities {init_states[i_state]} ($\phi_0: ${best_phis_avg_opt[0]:0.4})')
            plt.xlabel(f'phis Q{tomo_qubits[2]}')
            plt.ylabel(f'phis Q{tomo_qubits[1]}')
            plt.pcolormesh(x_sweep, y_sweep, fids_grids[i_state][best_phi_indices[0], :, :], cmap='viridis', shading='auto')
            plt.colorbar()
            plt.clim(vmin=0, vmax=vmax)
            plt.show()


    for i_state, init_state in enumerate(init_states):
        if init_state not in check_init_states: continue
        best_fids_avg_opt.append(fids_grids[i_state][best_phi_indices[0], best_phi_indices[1], best_phi_indices[2]])

    print('check states', check_init_states)
    print('Best fids with individual rotations', individual_best_fids.tolist())
    print('Avg of best fids with individual rotations', np.average(individual_best_fids))
    print('Phis for best fids with individual rotations', individual_best_phis.tolist())

    # test_indices = individual_best_phis_indices[0]
    # print(test_indices)
    # print(fids_grids[0][test_indices[0], test_indices[1], test_indices[2]], "should be", individual_best_fids[0])
    print('Best avg fid with overall optimal phis', best_avg_fid_avg_opt)
    print('Fids for check states using overall optimal phis', best_fids_avg_opt)
    print('Overall optimal phis', best_phis_avg_opt)
    return individual_best_fids, individual_best_phis, best_avg_fid_avg_opt, best_fids_avg_opt, best_phis_avg_opt

In [111]:
get_optimized_phis(get_fids_grids(os.path.join(data_path, fids_grids_filepath_5protocol)), use_init_2q_state=False, use_ZZ_correction=True, show_fid_grids=True) # ZZ_individual_vs_ideal

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_250119\202502121025_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 3, 4]


 C:\Users\slab\AppData\Local\Temp\ipykernel_40904\1035536604.py: 51

check states ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
Best fids with individual rotations [0.9573333128353031, 0.8552540145736229, 0.8704031305691078, 0.9214954309775907, 0.8296395979355672, 0.8005522854387742, 0.7016429725736425, 0.794747399195225, 0.8776610779042301, 0.7131969243692221, 0.737799611082338, 0.7625331444498756, 0.8892889558979387, 0.7394061026984511, 0.7437975549776789, 0.7994896218498727]
Avg of best fids with individual rotations 0.8121400710830275
Phis for best fids with individual rotations [[170.52631578947367, 246.3157894736842, 170.52631578947367], [75.78947368421052, 208.42105263157893, 18.94736842105263], [56.84210526315789, 94.73684210526315, 18.94736842105263], [75.78947368421052, 113.68421052631578, 170.52631578947367], [341.05263157894734, 284.2105263157895, 18.94736842105263], [303.1578947368421, 0.0, 113.68421052631578

(array([0.95733331, 0.85525401, 0.87040313, 0.92149543, 0.8296396 ,
        0.80055229, 0.70164297, 0.7947474 , 0.87766108, 0.71319692,
        0.73779961, 0.76253314, 0.88928896, 0.7394061 , 0.74379755,
        0.79948962]),
 array([[170.52631579, 246.31578947, 170.52631579],
        [ 75.78947368, 208.42105263,  18.94736842],
        [ 56.84210526,  94.73684211,  18.94736842],
        [ 75.78947368, 113.68421053, 170.52631579],
        [341.05263158, 284.21052632,  18.94736842],
        [303.15789474,   0.        , 113.68421053],
        [208.42105263, 246.31578947, 208.42105263],
        [246.31578947, 284.21052632, 189.47368421],
        [341.05263158, 303.15789474, 208.42105263],
        [ 94.73684211,   0.        ,  56.84210526],
        [341.05263158,  56.84210526, 189.47368421],
        [341.05263158,  75.78947368, 189.47368421],
        [341.05263158, 113.68421053, 227.36842105],
        [  0.        , 284.21052632,  56.84210526],
        [341.05263158,  75.78947368, 189.47368

In [112]:
def get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis):
    return 'fidelity_' + ('ZZ' if use_ZZ_correction else 'base') + '_rotated_' + ('individual' if not use_avg_opt_phis else 'opt') + '_vs_' + ('meas' if use_init_2q_state else 'ideal')

def get_rho_qram(fids_grids_dict, rho_qram, use_init_2q_state, use_ZZ_correction, individual_best_phis, best_phis_avg_opt):
    init_states = fids_grids_dict["init_states"]

    z_phi123_avg_opt = tomo_analysis.z_gate_nq(best_phis_avg_opt)

    for use_avg_opt_phis in [True, False]:
        fidelity_name = get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis)
        print(f'Calculating {fidelity_name}')
        for i_state, init_state in enumerate(init_states):
            if use_avg_opt_phis:
                z_phi123 = z_phi123_avg_opt
            else:
                z_phi123 = tomo_analysis.z_gate_nq(individual_best_phis[i_state])

            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()

            rho_id = get_qram_qSLR_state_from_rho(init_rho)

            n_tomo_corrected = fids_grids_dict[init_state]

            rho_MLE = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=False,
                method='analytical',
                ZZ_correction=use_ZZ_correction,
                evol_mats=evol_mats,
            )

            rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
            fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2

            rho_MLE_name = 'rho_' + ('ZZ' if use_ZZ_correction else 'base')
            rho_qram[rho_MLE_name].update({init_state:rho_MLE_rot})
            rho_qram[fidelity_name][init_state] = fid_rot
        print(rho_qram[fidelity_name])
        # rho_qram['avg_'+fidelity_name] = np.average(list(rho_qram[fidelity_name].values()))
    return rho_qram

In [113]:
rho_qrams = []
for fids_grids_filepath in fids_grids_filepaths:
    print()
    rho_qram = {
        'rho_base': {},
        'rho_ZZ': {},

        'fidelity_base_rotated_individual_vs_meas': {},
        'fidelity_base_rotated_individual_vs_ideal': {},
        'fidelity_ZZ_rotated_individual_vs_meas': {},
        'fidelity_ZZ_rotated_individual_vs_ideal': {},
        'fidelity_base_rotated_opt_vs_meas': {},
        'fidelity_base_rotated_opt_vs_ideal': {},
        'fidelity_ZZ_rotated_opt_vs_meas': {},
        'fidelity_ZZ_rotated_opt_vs_ideal': {},
        }
    fids_grids_dict = get_fids_grids(os.path.join(data_path, fids_grids_filepath))
    for use_init_2q_state in [True, False]:
        for use_ZZ_correction in [True, False]:
            individual_best_fids, individual_best_phis, best_avg_fid_avg_opt, best_fids_avg_opt, best_phis_avg_opt = get_optimized_phis(fids_grids_dict, use_init_2q_state, use_ZZ_correction, show_fid_grids=False)
            rho_qram = get_rho_qram(fids_grids_dict, rho_qram, use_init_2q_state, use_ZZ_correction, individual_best_phis, best_phis_avg_opt)
            print()
    rho_qrams.append(rho_qram)
# print(rho_qram)


Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_250119\202502120248_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 3, 4]
check states ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
Best fids with individual rotations [0.9789081726211734, 0.9484819913285831, 0.9562234784767824, 0.9725722873910322, 0.9711892037033085, 0.8853536674578224, 0.916599037498118, 0.9759699465553998, 0.9729434080708766, 0.9583762849597928, 0.9768011702591339, 0.9682300897914508, 0.9757626749205368, 0.9360473888370697, 0.9618769240203576, 0.9735274406207944]
Avg of best fids with individual rotations 0.9580539479070145
Phis for best fids with

 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1178ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1183

{'|0>|0>': 0.9787024420164843, '|0>|1>': 0.9482316081080275, '|0>|0+1>': 0.9535912579072606, '|0>|0+i>': 0.9672962569970011, '|1>|0>': 0.9706847038571165, '|1>|1>': 0.8816633914984839, '|1>|0+1>': 0.9136708509064848, '|1>|0+i>': 0.9757103408214634, '|0+1>|0>': 0.9724847369197368, '|0+1>|1>': 0.954047278293774, '|0+1>|0+1>': 0.974673281030588, '|0+1>|0+i>': 0.9669789931413183, '|0+i>|0>': 0.9750182325799325, '|0+i>|1>': 0.9313142571426318, '|0+i>|0+1>': 0.9575882342636909, '|0+i>|0+i>': 0.9716676521528796}
Calculating fidelity_ZZ_rotated_individual_vs_meas
{'|0>|0>': 0.9789081726211734, '|0>|1>': 0.9484819913285831, '|0>|0+1>': 0.9562234784767824, '|0>|0+i>': 0.9725722873910322, '|1>|0>': 0.9711892037033085, '|1>|1>': 0.8853536674578224, '|1>|0+1>': 0.916599037498118, '|1>|0+i>': 0.9759699465553998, '|0+1>|0>': 0.9729434080708766, '|0+1>|1>': 0.9583762849597928, '|0+1>|0+1>': 0.9768011702591339, '|0+1>|0+i>': 0.9682300897914508, '|0+i>|0>': 0.9757626749205368, '|0+i>|1>': 0.936047388837

# Plotting

## Decay of fidelity over number of protocols

Average over init states, binned into cqs

In [30]:
use_fid = 'fidelity_base_rotated_individual_vs_meas'
check_init_states = None # set to None to average over all states, otherwise will only average over specified states

In [31]:
x_sweep = [0, 1, 2, 3]

CQ_fids_dict_all_files = dict(
    CsCi=[], # average fid for each state in this category from all files
    CsQi=[],
    QsCi=[],
    QsQi=[],
)

for i_file, rho_qram in enumerate(rho_qrams):
    CQ_fids_dict = dict(
        CsCi=[], # fid for each state in this category from this file
        CsQi=[],
        QsCi=[],
        QsQi=[],
    )
    fids = rho_qram[use_fid] # dict: init_state -> fidelity for given number of protocols
    print(fids)
    for i, init_state in enumerate(fids.keys()):
        if check_init_states is not None and init_state not in check_init_states: continue
        cq = CQ_from_init(init_state)
        CQ_fids_dict[cq].append(fids[init_state])
        print(f"from file {i_file} adding fid {fids[init_state]} of {init_state} to {cq}")
    for cq in CQ_fids_dict.keys():
        CQ_fids_dict_all_files[cq].append(np.average(CQ_fids_dict[cq]))

{'|0>|0>': 0.9830631224672989, '|0>|1>': 0.9523573681917928, '|0>|0+1>': 0.9577270887388704, '|0>|0+i>': 0.9834979262024689, '|1>|0>': 0.973783648783532, '|1>|1>': 0.9705762515776071, '|1>|0+1>': 0.929571839234933, '|1>|0+i>': 0.9763918198484435, '|0+1>|0>': 0.9787326946232898, '|0+1>|1>': 0.9909793535412924, '|0+1>|0+1>': 0.9788873683207339, '|0+1>|0+i>': 0.9678650221598891, '|0+i>|0>': 0.9761687388938209, '|0+i>|1>': 0.9835060689497167, '|0+i>|0+1>': 0.967239688022744, '|0+i>|0+i>': 0.9694294411362802}
from file 0 adding fid 0.9830631224672989 of |0>|0> to CsCi
from file 0 adding fid 0.9523573681917928 of |0>|1> to CsCi
from file 0 adding fid 0.9577270887388704 of |0>|0+1> to CsQi
from file 0 adding fid 0.9834979262024689 of |0>|0+i> to CsQi
from file 0 adding fid 0.973783648783532 of |1>|0> to CsCi
from file 0 adding fid 0.9705762515776071 of |1>|1> to CsCi
from file 0 adding fid 0.929571839234933 of |1>|0+1> to CsQi
from file 0 adding fid 0.9763918198484435 of |1>|0+i> to CsQi
from

In [96]:
saveplot = True

# fig, ax = plt.subplots(1, 1, figsize=(5, 3))
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))
plt.grid()

for cq in CQ_fids_dict_all_files.keys():
    print(cq)
    fids_per_U = CQ_fids_dict_all_files[cq]
    # err_per_U = np.diff(fids_per_U)
    # avg_err_per_U = np.average(np.abs(err_per_U))/2

    denominator = np.roll(fids_per_U, 1)[1:]
    gate_fids_per_U2 = fids_per_U[1:] / denominator
    gate_fids_per_U = np.sqrt(gate_fids_per_U2)
    err_per_U = 1 - gate_fids_per_U
    avg_err_per_U = np.average(np.abs(err_per_U))
    print("denominator", denominator)

    print("fids_per_U", fids_per_U)
    print("err_per_U", err_per_U)
    print("avg_err_per_U", avg_err_per_U)
    plt.plot(range(len(fids_per_U)), fids_per_U, 'o-', label="$\epsilon$="+f"{avg_err_per_U*100:.2}%", color=color_from_cq(cq), markersize=5)

ax.set_xticks(range(len(fids_per_U)))
ymin = 0.6
ymax = 1.0
ytick_spacing = 0.1
ytick_labels = np.round(np.linspace(ymin, ymax, int(np.ceil((ymax-ymin)/ytick_spacing))+1) /ytick_spacing) * ytick_spacing
ax.set_yticks(ytick_labels)
ax.tick_params(axis='both', which='major', labelsize=16)
plt.xlabel('$N$ for $(U_{\mathrm{route}}U_{\mathrm{route}}^{-1})^N U_{\mathrm{route}}$', fontsize=16)
plt.ylabel('Average Fidelity', fontsize=16)
plt.ylim(ymin, ymax)
plt.legend(fontsize=14, loc='lower left', frameon=False)
plt.tight_layout()

fids_grids_filepaths_str = '_'.join([fids_grids_filepath[:12] for fids_grids_filepath in fids_grids_filepaths])
filename = f'protocol_repeat_{use_fid}_{fids_grids_filepaths_str}'
print(filename)
if saveplot:
    savename = os.path.join(save_plot_path, filename+'.svg')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved svg plot", savename)

    savename = os.path.join(save_plot_path, filename+'.pdf')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved pdf plot", savename)

    plt.close()
else:
    plt.show()

CsCi
denominator [0.9699451  0.953066   0.90796499 0.88689184 0.88239007]
fids_per_U [0.9699450977550577, 0.9530659996188611, 0.9079649867813897, 0.8868918437069699, 0.8823900721109446, 0.8970819777638732]
err_per_U [ 0.00873925  0.02394776  0.01167273  0.00254118 -0.0082907 ]
avg_err_per_U 0.011038321097453419
CsQi
denominator [0.96179717 0.91454396 0.89301807 0.81951325 0.85152845]
fids_per_U [0.9617971685061789, 0.9145439633864539, 0.8930180724819663, 0.8195132511605134, 0.8515284538606012, 0.7910024468351491]
err_per_U [ 0.02487443  0.01183873  0.04203892 -0.01934593  0.03619465]
avg_err_per_U 0.026858529798346596
QsCi
denominator [0.98234671 0.95296673 0.87350112 0.83673856 0.84924611]
fids_per_U [0.98234671400203, 0.9529667333514926, 0.8735011175122105, 0.8367385645202423, 0.8492461089310636, 0.7877910847781444]
err_per_U [ 0.01506749  0.04260124  0.02126941 -0.00744626  0.03686149]
avg_err_per_U 0.024649178988553032
QsQi
denominator [0.97085538 0.89603966 0.8225735  0.75655892 0

## Error breakdown

In [114]:
x_sweep = [0, 1, 2, 3]

i_file = 0 # index in file list to use the data from

CQ_fids_dict_processing_dict = dict() # dict: use_fid -> CQ -> average fid
for use_init_2q_state in [True, False]:
    for use_ZZ_correction in [True, False]:
        for use_avg_opt_phis in [True, False]:
            use_fid = get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis)

            CQ_fids_dict = dict(
                CsCi=[], # fid for each state in this category from this file
                CsQi=[],
                QsCi=[],
                QsQi=[],
            )

            rho_qram = rho_qrams[i_file]
            fids = rho_qram[use_fid] # dict: init_state -> fidelity for given number of protocols
            # print(fids)
            for i, init_state in enumerate(fids.keys()):
                if check_init_states is not None and init_state not in check_init_states: continue
                cq = CQ_from_init(init_state)
                CQ_fids_dict[cq].append(fids[init_state])
                # print(f"from file {i_file} adding fid {fids[init_state]} of {init_state} to {cq}")
            for cq in CQ_fids_dict.keys():
                CQ_fids_dict[cq] = np.average(CQ_fids_dict[cq])

            CQ_fids_dict_processing_dict[use_fid] = CQ_fids_dict

for key in CQ_fids_dict_processing_dict.keys():
    print(key)
    print(CQ_fids_dict_processing_dict[key])

fidelity_ZZ_rotated_opt_vs_meas
{'CsCi': 0.944820536370028, 'CsQi': 0.9525671766580525, 'QsCi': 0.9582161262340188, 'QsQi': 0.9677270401471193}
fidelity_ZZ_rotated_individual_vs_meas
{'CsCi': 0.9459832587777219, 'CsQi': 0.9553411874803331, 'QsCi': 0.960782439197069, 'QsQi': 0.9701089061729342}
fidelity_base_rotated_opt_vs_meas
{'CsCi': 0.9464248365511895, 'CsQi': 0.9517841081657334, 'QsCi': 0.9582999237101559, 'QsQi': 0.970697631095615}
fidelity_base_rotated_individual_vs_meas
{'CsCi': 0.9474333619150656, 'CsQi': 0.9547076052094744, 'QsCi': 0.9600052934563066, 'QsQi': 0.9732698685979467}
fidelity_ZZ_rotated_opt_vs_ideal
{'CsCi': 0.9461350014794122, 'CsQi': 0.9311950536873518, 'QsCi': 0.9434253847898995, 'QsQi': 0.9026011534993676}
fidelity_ZZ_rotated_individual_vs_ideal
{'CsCi': 0.946137585983253, 'CsQi': 0.9311979125420335, 'QsCi': 0.9471772499220661, 'QsQi': 0.9041206816326639}
fidelity_base_rotated_opt_vs_ideal
{'CsCi': 0.9464621852609121, 'CsQi': 0.9338467314829506, 'QsCi': 0.94278

Fidelities bar chart

In [115]:
saveplot = True

# fig, ax = plt.subplots(1, 1, figsize=(5, 2.5))
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3))
cqs = ['CsCi', 'CsQi', 'QsCi', 'QsQi']
cq_labels = [latex_from_cq(cq) for cq in cqs]

use_ZZ_correction = True

# use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=False, use_avg_opt_phis=True)
# fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
fid_theory = dict(
    CsCi = 0.9795465754354343,
    QsCi = 0.97274267574603,
    CsQi = 0.9727978474810874,
    QsQi = 0.9660012105228144,
)
fids = [fid_theory[cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle='-', fill=False, label='Theory')

# use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=False, use_avg_opt_phis=True)
# fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
# colors_cq = [color_from_cq(cq) for cq in cqs]
# ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=0.75, linestyle='--', fill=False, label='Base, $\\vec{\phi}_0$')

use_fid = get_fid_name(use_init_2q_state=False, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
label = '$\\rho_{\mathrm{ideal}}$, $\\vec{\phi}_0$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=0.5, edgecolor=colors_cq, linewidth=0.0, fill=True, label=label)

use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
label = '$\\rho_{\mathrm{meas}}$, $\\vec{\phi}_0$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle='--', fill=False, label=label)

use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=False)
label = '$\\rho_{\mathrm{meas}}$, $\{ \\vec{\phi}_k\}$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle=':', fill=False, label=label)


ymin = 0.80
ymax = 1.01
ytick_spacing = 0.1
ytick_labels = np.round(np.linspace(ymin, ymax, int(np.ceil((ymax-ymin)/ytick_spacing))+1) /ytick_spacing) * ytick_spacing
ax.set_yticks(ytick_labels)
ax.tick_params(axis='x', which='major', labelsize=16)
ax.tick_params(axis='y', which='major', labelsize=16)
# for tick in ax.get_xticklabels(): # tilt the x labels by 45 degrees
    # tick.set_rotation(45)
plt.ylabel('Average Fidelity', fontsize=16)
plt.ylim(ymin, ymax)
# plt.legend(bbox_to_anchor=(1.0, 0.5), loc='center left', frameon=True, edgecolor='k', fontsize=13) # bbox is where to put box, loc is which corner of box is anchored
plt.legend(bbox_to_anchor=(0.45, -0.1), loc='upper center', frameon=True, edgecolor='k', fontsize=16, ncol=2) # bbox is where to put box, loc is which corner of box is anchored
# plt.tight_layout()

fids_grids_filepaths_str = fids_grids_filepaths[0][:12]
filename = f'cq_fid_breakdown_1protocol_{fids_grids_filepaths_str}'
print(filename)
if saveplot:
    savename = os.path.join(save_plot_path, filename+'.svg')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved svg plot", savename)

    savename = os.path.join(save_plot_path, filename+'.pdf')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved pdf plot", savename)

    plt.close()
else:
    plt.show()

cq_fid_breakdown_1protocol_202502120248
Saved svg plot S:\QRAM\qram_4QR2\plots\paper\cq_fid_breakdown_1protocol_202502120248.svg
Saved pdf plot S:\QRAM\qram_4QR2\plots\paper\cq_fid_breakdown_1protocol_202502120248.pdf


Errors bar chart

In [117]:
saveplot = True

# fig, ax = plt.subplots(1, 1, figsize=(5, 2.5))
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3))
# plt.grid(axis='y')
cqs = ['CsCi', 'CsQi', 'QsCi', 'QsQi']
cq_labels = [latex_from_cq(cq) for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
rgbcolors_cq = [mcolors.to_rgb(c) for c in colors_cq]

use_ZZ_correction = True

bottom = np.zeros(len(cqs))

use_fid = get_fid_name(use_init_2q_state=False, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
fids_id_global = np.array([CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs])
use_fid = get_fid_name(use_init_2q_state=False, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=False)
fids_id_individual = np.array([CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs])
use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
fids_meas_global = np.array([CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs])
use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=False)
fids_meas_individual = np.array([CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs])

label = "Total Error"
errs = 1 - fids_id_global
ax.bar(cq_labels, errs, alpha=0.6, edgecolor='k', linewidth=2.0, linestyle='-', fill=False, label=label, bottom=np.zeros(len(cqs)))

# use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=False, use_avg_opt_phis=True)
# fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
label = "Decoherence\n(Theory)"
errs = [1 - fid_theory[cq] for cq in cqs]
facecolors = [(*rgbcolors_cq[c], 0.25) for c in range(len(cqs))]
edgecolors = [(*rgbcolors_cq[c], 1.0) for c in range(len(cqs))]
ax.bar(cq_labels, errs, color=facecolors, edgecolor=edgecolors, linewidth=2.0, fill=True, label=label, bottom=bottom)
bottom += errs

label = "Timing Errors"
errs = fids_id_individual - fids_id_global
facecolors = [(*rgbcolors_cq[c], 0.75) for c in range(len(cqs))]
edgecolors = [(*rgbcolors_cq[c], 1.0) for c in range(len(cqs))]
ax.bar(cq_labels, errs, color=facecolors, edgecolor=edgecolors, linewidth=2.0, fill=True, label=label, bottom=bottom)
bottom += errs

label = "Initial State\nSPAM"
errs = fids_meas_individual - fids_id_individual
facecolors = [(*rgbcolors_cq[c], 0.0) for c in range(len(cqs))]
edgecolors = [(*rgbcolors_cq[c], 1.0) for c in range(len(cqs))]
ax.bar(cq_labels, errs, color=facecolors, edgecolor=edgecolors, linewidth=2.0, hatch='x', label=label, bottom=bottom)
bottom += errs


ymin = 0.0
ymax = 0.15
ytick_spacing = 0.05
ytick_labels = np.round(np.linspace(ymin, ymax, int(np.ceil((ymax-ymin)/ytick_spacing))+1) /ytick_spacing) * ytick_spacing
ax.set_yticks(ytick_labels)
ax.tick_params(axis='x', which='major', labelsize=16)
ax.tick_params(axis='y', which='major', labelsize=16)
# for tick in ax.get_xticklabels(): # tilt the x labels by 45 degrees
    # tick.set_rotation(45)
plt.ylabel('Error Contribution', fontsize=16)
plt.ylim(ymin, ymax)
# plt.legend(bbox_to_anchor=(1.0, 0.5), loc='center left', frameon=True, edgecolor='k', fontsize=13) # bbox is where to put box, loc is which corner of box is anchored
plt.legend(bbox_to_anchor=(0.45, -0.1), loc='upper center', frameon=True, edgecolor='k', fontsize=16, ncol=2) # bbox is where to put box, loc is which corner of box is anchored
# plt.tight_layout()

errs_grids_filepaths_str = fids_grids_filepaths[0][:12]
filename = f'cq_err_breakdown_1protocol_{errs_grids_filepaths_str}'
print(filename)
if saveplot:
    savename = os.path.join(save_plot_path, filename+'.svg')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved svg plot", savename)

    savename = os.path.join(save_plot_path, filename+'.pdf')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved pdf plot", savename)

    plt.close()
else:
    plt.show()

cq_err_breakdown_1protocol_202502120248
Saved svg plot S:\QRAM\qram_4QR2\plots\paper\cq_err_breakdown_1protocol_202502120248.svg
Saved pdf plot S:\QRAM\qram_4QR2\plots\paper\cq_err_breakdown_1protocol_202502120248.pdf
